# Image Classification Problem - Classify Cat vs Dog

## Data Pre-processing

In [1]:
#code from keras documentation - https://keras.io/preprocessing/image/
import os
from keras.preprocessing.image import ImageDataGenerator
#Image augmentation - rotate, shift, flip, etc.. copies of image to expand size of training set
#rescale in a range from zero to 1

train_data_generator = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_data_generator.flow_from_directory('dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory('dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Using TensorFlow backend.


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Convolutional Neural Network implementation

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

#Initializing the Convolutional Neural Network

def build_cnn():
    #Step 1 - Convolution
    classifier = Sequential()

    #add feature detector number of rows and columns as kernel size parameter
    #image 64 x 64 with 3 color channels
    #relu activation function to remove linearity
    cnn_input_layer = Conv2D(filters = 32, kernel_size = (3, 3), input_shape = (64, 64, 3), activation = 'relu')
    classifier.add(cnn_input_layer)
    #Step 2 - Max Pooling

    #reduce size of feature maps using max pooling
    max_pooling_layer = MaxPooling2D(pool_size = (2, 2))
    classifier.add(max_pooling_layer)
    
    #Create Deep Fully Connected Layers (hidden layers) - 2nd Layer
    fcl = Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu')
    classifier.add(fcl)
    classifier.add(MaxPooling2D(pool_size = (2, 2)))

    #Step 3 - Flattening
    flattening_layer = Flatten()
    classifier.add(flattening_layer)

    #Step 4 - Full Connection
    ann_fully_connected_layer = Dense(units = 128, activation = 'relu') #same as ANN hidden layer
    classifier.add(ann_fully_connected_layer)

    ann_output_layer = Dense(units = 1, activation = 'sigmoid')
    classifier.add(ann_output_layer)
    return classifier

## Fitting CNN

In [3]:
classifier = build_cnn()
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [4]:
classifier.fit_generator(generator = train_generator, steps_per_epoch = 8000, epochs = 25, validation_data = test_generator, workers = 50, verbose=1)

Epoch 1/25
8000/8000 [==============================] - 1006s 126ms/step - loss: 0.3419 - accuracy: 0.8394 - val_loss: 0.0523 - val_accuracy: 0.7920
Epoch 2/25
8000/8000 [==============================] - 751s 94ms/step - loss: 0.0868 - accuracy: 0.9679 - val_loss: 0.7075 - val_accuracy: 0.8085
Epoch 3/25
8000/8000 [==============================] - 772s 96ms/step - loss: 0.0470 - accuracy: 0.9832 - val_loss: 1.8699 - val_accuracy: 0.7945
Epoch 4/25
8000/8000 [==============================] - 902s 113ms/step - loss: 0.0351 - accuracy: 0.9881 - val_loss: 1.0068 - val_accuracy: 0.7950
Epoch 5/25
8000/8000 [==============================] - 1147s 143ms/step - loss: 0.0282 - accuracy: 0.9905 - val_loss: 0.1689 - val_accuracy: 0.8025
Epoch 6/25
8000/8000 [==============================] - 1103s 138ms/step - loss: 0.0251 - accuracy: 0.9917 - val_loss: 1.8171 - val_accuracy: 0.8030
Epoch 7/25
8000/8000 [==============================] - 1210s 151ms/step - loss: 0.0213 - accuracy: 0.9929 - va

## Making new prediction

In [8]:
import numpy as np
from keras.preprocessing import image

#Load Image
dog_or_cat_test1 = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
dog_or_cat_test1 = image.img_to_array(dog_or_cat_test1)

#add dimension for neural network prediction
dog_or_cat_test1 = np.expand_dims(dog_or_cat_test1, axis = 0) #added to beginning of array
#make prediction
binary_result = classifier.predict(dog_or_cat_test1)
binary_result

array([[1.]], dtype=float32)

In [9]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}